In [2]:
!pip install --upgrade pip         # pip 19.0 or higher is required for TF 2
!pip install --upgrade setuptools
!pip install --user --upgrade tensorflow

     |████████████████████████████████| 1.5MB 5.6MB/s eta 0:00:01
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 789 kB 3.1 MB/s eta 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 45.2.0
    Uninstalling setuptools-45.2.0:
      Successfully uninstalled setuptools-45.2.0
     |████████████████████████████████| 516.2 MB 5.7 kB/s  eta 0:00:01  |▍                               | 6.8 MB 5.0 MB/s eta 0:01:42     |██▋                             | 41.8 MB 5.0 MB/s eta 0:01:35     |█████▏                          | 82.8 MB 73.5 MB/s eta 0:00:06     |██████▌                         | 104.1 MB 49.3 MB/s eta 0:00:09     |████████████▋                   | 204.2 MB 47.7 MB/s eta 0:00:07     |███████████████                 | 241.2 MB 45.0 MB/s eta 0:00:07     |███████████████████████████▋    | 446.0 MB 38.6 MB/s eta 0:00:02     |█████████████████████████

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
!wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
!unzip wwm_uncased_L-24_H-1024_A-16.zip

!ls wwm_uncased_L-24_H-1024_A-16

!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

!ls uncased_L-12_H-768_A-12

!pip install simpletransformers
!pip install transformers
!pip install tensorboardx
!pip install ktrain

--2020-07-25 05:53:41--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.14.240, 216.58.193.80, 216.58.217.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.14.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1248381879 (1.2G) [application/zip]
Saving to: ‘wwm_uncased_L-24_H-1024_A-16.zip’

wwm_uncased_L-24_H- 100%[===================>]   1.16G  46.5MB/s    in 20s     

2020-07-25 05:54:01 (58.5 MB/s) - ‘wwm_uncased_L-24_H-1024_A-16.zip’ saved [1248381879/1248381879]

Archive:  wwm_uncased_L-24_H-1024_A-16.zip
   creating: wwm_uncased_L-24_H-1024_A-16/
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: wwm_uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating

In [3]:
!pip install bert
!pip install bert-tensorflow
!pip install tensorflow_hub
!pip install bert-for-tf2

  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3754 sha256=74a82a3ce0ba980b25538864137c4aa491ec968f3043544800a63786034fdbca
  Stored in directory: /home/ec2-user/.cache/pip/wheels/cb/da/90/bc3128c1676721ede415ed707464c4ea5671ee3e292e6b5404
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6787 sha256=418ef8425b7a2b7e97f3f9cd9055c0e6d47cbaf4be2249fb0c9b40c8e79759a5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/10/bb/e1/a8f26a46f7f4d9278a50c924b93a1664e95db12b0d785da5b4
Successfully built bert erlastic
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 67 kB 3.1 MB/s eta 0:00:011
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 101 kB 3.7 MB/s ta 0:00:011
You should consider upgrading v

In [19]:
import pandas as pd
import re
import string
import bert
import tensorflow_hub as hub
import numpy as np
from sklearn.model_selection import train_test_split
import os
#from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras import layers
import pickle
import math
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
%matplotlib inline
import random
import requests
import sys
import urllib.parse
import json

In [26]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)

In [27]:
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [28]:
def create_model(max_seq_len,cnn_filters,dropout_rate,dnn_units):
    input_ids = keras.layers.Input(
                                   shape=(max_seq_len, ),
                                   dtype='int32',
                                   name="input_ids"
                                   )
    input_mask = keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="input_mask")
    segment_ids = keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="segment_ids")
    
    _, bert_output = bert_layer([input_ids, input_mask, segment_ids])
    
    cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")(bert_output)
    cnn_layer1 = layers.GlobalMaxPool1D()(cnn_layer1)
    cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")(bert_output)
    cnn_layer2 = layers.GlobalMaxPool1D()(cnn_layer2)
    cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")(bert_output)
    cnn_layer3 = layers.GlobalMaxPool1D()(cnn_layer3)

    concatenated = tf.concat([cnn_layer1, cnn_layer2, cnn_layer3], axis=-1) # (batch_size, 3 * cnn_filters)   
    dense_1 = layers.Dense(units=dnn_units, activation="relu")(concatenated)
    dropout = layers.Dropout(rate=dropout_rate)(concatenated)
       
    last_dense = layers.Dense(units=1,activation="sigmoid")(dropout)
    
    model = keras.Model(inputs=[input_ids,input_mask,segment_ids], outputs=last_dense)
    
    model.build(input_shape=(None, max_seq_len))

    return model

In [10]:
import boto3
from sagemaker import get_execution_role

role = get_execution_role()
bucket='trainedbertmodelweights'
data_key = 'CNN_BERT_SIMILAR_TRAINABLE_V2_weights-improvement-06-0.98.hdf5'
weights_location = 's3://{}/{}'.format(bucket, data_key)
print(weights_location)

local_weights_file = 'weights.hdf5'
s3_client = boto3.client('s3')
s3_client.download_file(bucket, data_key, local_weights_file)

s3://trainedbertmodelweights/CNN_BERT_SIMILAR_TRAINABLE_V2_weights-improvement-06-0.98.hdf5


In [29]:
# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have","she's":"she is","he's":"he is"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

def preprocess_text(sen):
    #convert ’ to '
    
    sentence = sen.replace("’","'")
    
    #expand contractions 
    sentence = expand_contractions(sentence.lower())
    # Removing punctuation
    #sentence = re.sub('<[^>]+>',' ', sen)

    # Remove punctuations and numbers and foreign characters
    sentence = re.sub('[^a-zA-Z]',' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+",' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+',' ', sentence)

    return sentence

In [30]:
precision_obj = tf.keras.metrics.Precision()
recall_obj = tf.keras.metrics.Recall()

CNN_FILTERS = 100
DROPOUT_RATE = 0.2
DNN_UNITS = 256
MAX_SEQ_LEN = 202
model = create_model(MAX_SEQ_LEN,CNN_FILTERS,DROPOUT_RATE,DNN_UNITS)
#load model weights
model.load_weights("./weights.hdf5")

model.compile(
               optimizer=keras.optimizers.Adam(1e-5),
               loss='binary_crossentropy',
               run_eagerly=True,
               metrics=['accuracy',precision_obj,recall_obj]
             )

In [31]:
def predict_claims(claim_text):
    claims = [preprocess_text(claim_text)]

    max_seq_len=202
    x_input,masks,segments = [], [],[]

    for new_claim in claims:
        
        similar_claim1 = new_claim
        similar_claim2 = new_claim
        similar_claim3 = new_claim
        similar_claim4 = new_claim
        similar_claim5 = new_claim
        similar_claim6 = new_claim
        similar_claim7 = new_claim
        similar_claim8 = new_claim
        similar_claim9 = new_claim
        try:
            payload = {'claim': urllib.parse.quote(new_claim)}
            r = requests.get('https://88rrgid4rl.execute-api.us-west-2.amazonaws.com/similar-claims?claim='\
                             +urllib.parse.quote(new_claim)
                            )
            results = json.loads(r.text) 
            if results.get('claim') is not None:
                similar_claim1 = results.get("similar_claims")[0]['clean_claim']
                similar_claim2 = results.get("similar_claims")[1]['clean_claim']
                similar_claim3 = results.get("similar_claims")[2]['clean_claim']
                similar_claim4 = results.get("similar_claims")[3]['clean_claim']
                similar_claim5 = results.get("similar_claims")[4]['clean_claim']
                similar_claim6 = results.get("similar_claims")[5]['clean_claim']
                similar_claim7 = results.get("similar_claims")[6]['clean_claim']
                similar_claim8 = results.get("similar_claims")[7]['clean_claim']
                similar_claim9 = results.get("similar_claims")[8]['clean_claim']
            else: 
                #try one more time 
                payload = {'claim': urllib.parse.quote(new_claim)}
                r = requests.get('https://88rrgid4rl.execute-api.us-west-2.amazonaws.com/similar-claims?claim='\
                             +urllib.parse.quote(new_claim)
                            )
                results = json.loads(r.text) 
                if results.get('claim') is not None:
                    similar_claim1 = results.get("similar_claims")[0]['clean_claim']
                    similar_claim2 = results.get("similar_claims")[1]['clean_claim']
                    similar_claim3 = results.get("similar_claims")[2]['clean_claim']
                    similar_claim4 = results.get("similar_claims")[3]['clean_claim']
                    similar_claim5 = results.get("similar_claims")[4]['clean_claim']
                    similar_claim6 = results.get("similar_claims")[5]['clean_claim']
                    similar_claim7 = results.get("similar_claims")[6]['clean_claim']
                    similar_claim8 = results.get("similar_claims")[7]['clean_claim']
                    similar_claim9 = results.get("similar_claims")[8]['clean_claim']
        except:

            #try once more 
            try:
                payload = {'claim': urllib.parse.quote(new_claim)}
                r = requests.get('https://88rrgid4rl.execute-api.us-west-2.amazonaws.com/similar-claims?claim='\
                                 +urllib.parse.quote(new_claim)
                                )
                results = json.loads(r.text) 
                if results.get('claim') is not None:
                    similar_claim1 = results.get("similar_claims")[0]['clean_claim']
                    similar_claim2 = results.get("similar_claims")[1]['clean_claim']
                    similar_claim3 = results.get("similar_claims")[2]['clean_claim']
                    similar_claim4 = results.get("similar_claims")[3]['clean_claim']
                    similar_claim5 = results.get("similar_claims")[4]['clean_claim']
                    similar_claim6 = results.get("similar_claims")[5]['clean_claim']
                    similar_claim7 = results.get("similar_claims")[6]['clean_claim']
                    similar_claim8 = results.get("similar_claims")[7]['clean_claim']
                    similar_claim9 = results.get("similar_claims")[8]['clean_claim']
            except:

                 e = sys.exc_info()[0]

        
        text = tokenizer.tokenize(new_claim)

        similar_claim1 = tokenizer.tokenize(similar_claim1)
        similar_claim2 = tokenizer.tokenize(similar_claim2)
        similar_claim3 = tokenizer.tokenize(similar_claim3)
        similar_claim4 = tokenizer.tokenize(similar_claim4)
        similar_claim5 = tokenizer.tokenize(similar_claim5)
        similar_claim6 = tokenizer.tokenize(similar_claim6)
        similar_claim7 = tokenizer.tokenize(similar_claim7)
        similar_claim8 = tokenizer.tokenize(similar_claim8)
        similar_claim9 = tokenizer.tokenize(similar_claim9)

        text = text[:20]
        similar_claim1 = similar_claim1[:20]
        similar_claim2 = similar_claim2[:20]
        similar_claim3 = similar_claim3[:20]
        similar_claim4 = similar_claim4[:20]
        similar_claim5 = similar_claim5[:20]
        similar_claim6 = similar_claim6[:20]
        similar_claim7 = similar_claim7[:20]
        similar_claim8 = similar_claim8[:20]
        similar_claim9 = similar_claim9[:20]
        input_sequence = ["[CLS]"] +\
                         text +\
                         similar_claim1 +\
                         similar_claim2 +\
                         similar_claim3 +\
                         similar_claim4 +\
                         similar_claim5 +\
                         similar_claim6 +\
                         similar_claim7 +\
                         similar_claim8 +\
                         similar_claim9 +\
                         ["[SEP]"]

        
        
        pad_len = max_seq_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_seq_len
        x_input.append(np.array(tokens))
        masks.append(np.array(pad_masks))
        segments.append(np.array(segment_ids))

    return model.predict(x = [x_input,masks,segments])[0]

In [35]:
predict_claims('N-95 masks work better than cloth masks')[0]

0.4098307

In [34]:
print(predict_claims("Washing your hands can help keep you safe from coronavirus")[0])

0.99617755


In [39]:
predict_claims('Coronavirus will die off in the summer once it gets warmer')[0]

0.00012745535

In [37]:
predict_claims('Only certain face masks are effective and others, such as cloth masks, are not')[0]

0.084821686

In [22]:
new_claim="masks don't work"
payload = {'claim': urllib.parse.quote(new_claim)}
r = requests.get('https://88rrgid4rl.execute-api.us-west-2.amazonaws.com/similar-claims?claim='\
                 +urllib.parse.quote(new_claim)
                )

In [24]:
r.content

b'{"claim": "masks don\'t work", "sentiment": -0.800000011920929, "similar_claims": [{"claim": "Says only certain face masks are effective and others, such as cloth masks, are not.", "label": "false", "source_label": "false", "source": "Politifact", "date": "2020-04-02", "claim_source": "Facebook posts", "explanation": "", "fact_check_url": "https://politifact.com/factchecks/2020/may/18/facebook-posts/face-masks-including-homemade-ones-are-effective-c/", "clean_claim": "Says only certain face masks are effective and others such as cloth masks are not", "sentiments": -0.5, "cosine_dist": 0.26709584259787456, "sentiment_delta": 0.30000001192092896}, {"claim": "Only certain face masks are effective and others, such as cloth masks, are not.", "label": "false", "source_label": "false", "source": " PolitiFact", "date": "2020-04-02", "claim_source": " Facebook posts", "explanation": " N95 masks offer the most protection from viral particles, health officials say, but they should be reserved f